# Probando reglas de asociación

Dentro del aprendizaje no supervisado hay una familia de algoritmos que se suele denominar **Associative Learning Algorithms**, e incluyen a los siguientes:

* Apriori Algorithm
* Equivalence Classification Algorithm (Eclat)
* PrefixSpan
* FP-Growth

Todos ellos tienen en común el que se usan para buscar combinaciones de patrones en un conjunto de datos. El algoritmo APriori es probablemente el más conocido de todos ellos.

## El algoritmo APriori

El algoritmo APriori es un algoritmo de asociación clásico que se utiliza para buscar asociaciones entre elementos en una base de datos de transacciones.

Originariamente se aplicó a buscar asociaciones entre productos comprados en supermercados, por lo que a algoritmos similiares se les llama a veces también **market basket analysis**. Las transacciones serían cada una de las compras de un cliente, es decir, los productos que un cliente ha comprado en una visita al supermercado. Dicho de otro modo, su "ticket de la compra".



Los patrones se expresan habitualmente como **conjuntos frecuentes** (*frequent itemsets*). Son simplemente un conjunto de items que aparecen con frecuencia juntos en las transacciones. Por ejemplo, "lechuga" y "leche de soja" si se compran juntos con frecuencia.

Las **reglas de asociación** expresan relaciones fuertes entre esos conjuntos frecuentes e items. Por ejemplo, si alguien compra "cerveza" es bastante probable que también compre "pañales". Pero también se puede aplicar a otros contextos, por ejemplo en una red social si alguien "pulsa like" es muy probable que también "se suscriba". En este caso, los items son las acciones de los usuarios, y las transacciones conjuntos de acciones que hace un usuario en una sesión, por ejemplo. 

## Soporte y confianza

¿Cómo se decide qué relaciones (reglas) son realmente interesantes? Hay dos parámetros que definen la relevancia:
* El **soporte** de un itemset, que es el porcentaje de las transacciones del dataset que contiene ese itemset. 
* La **confianza** de una regla de asociación R del tipo $A \rightarrow B$ se define de la siguiente forma: $\frac{soporte(A, B)}{soporte(B)}$. Expresa la proporción de veces de nuestra base de datos en la que se da la asociación.

Las regla $A \rightarrow B$ [soporte=20%, confianza=70%] se interpreta así: *"Si el cliente compró el producto A, también compró el producto B en el 70% de los casos. Esto sucede en un 20% de los casos analizados."*. 

## Una implementación en Python

El algoritmo Apriori no está implementado en scikit-learn de momento. Su uso no encaja con las categorías de algoritmos de la biblioteca, por lo que se ha considerado que debería implementarse en otro paquete separado. 

Hay algunas implementaciones sencillas disponibles en Internet. Aquí vamos a usar una implementación de Everaldo Aguiar & Reid Johnson que podéis encontrar aquí:
http://nbviewer.ipython.org/github/cse40647/cse40647/blob/sp.14/10%20-%20Apriori.ipynb

In [2]:
import apriori

El algoritmo toma como entrada un conjunto de transacciones, que serían un conjunto de "listas de la compra", es decir, productos que un cliente ha comprado juntos en una visita al supermercado.
El algoritmo necesita como entrad una lista de conjuntos (set), lo primero que hacemos es preparar los datos.

En nuestro caso, los datos se representan como listas de listas en Python.

In [3]:

dataset =  [['Bread', 'Milk'], 
            ['Bread', 'Diapers', 'Beer', 'Eggs'], 
            ['Milk', 'Diapers', 'Beer', 'Coke'], 
            ['Bread', 'Milk', 'Diapers', 'Beer'], 
            ['Bread', 'Milk', 'Diapers', 'Coke'],
            ['Bread', 'Milk', 'Diapers'],
            ['Bread', 'Coke']]


Y simplemente pasamos el dataset al algoritmo. El parámetro fundamental que le tenemos que indicar es el **soporte mínimo**.

El algoritmo nos devuelve F que son los k-itemsets frecuentes (combinaciones de items o productos en nuestro caso que se dan con una frecuencia que al menos llega al soporte mínimo indicado), y también el soporte de cada uno de ellos. 

In [4]:
F, soporte = apriori.apriori(dataset, min_support=0.4, verbose=True)

{Coke}:  sup = 0.429
{Milk}:  sup = 0.714
{Bread}:  sup = 0.857
{Beer}:  sup = 0.429
{Diapers}:  sup = 0.714
{Beer, Diapers}:  sup = 0.429
{Diapers, Bread}:  sup = 0.571
{Milk, Diapers}:  sup = 0.571
{Milk, Bread}:  sup = 0.571
{Diapers, Milk, Bread}:  sup = 0.429


El segundo paso es generar las reglas a partir de los k-itemsets frecuentes. En este segundo paso es cuando hay que especificar la **confianza mínima** que consideramos para que una asociación sea significativa.

In [5]:
# Generamos las reglas de asociación de la lista de itemsets frecuentes
# En este caso, se filtran las que no llegan a una confianza determinada.
H = apriori.generate_rules(F, soporte, min_confidence=0.9, verbose=True)

{Beer} ---> {Diapers}:  conf = 1.0, sup = 0.429


Vemos en este caso que solo se encuentra una asociación fuerte para un soporte del 40% y una confianza del 80%. Concretamente, siempre que se compra cerveza se compran pañales. 

Lógicamente, si rebajamos la confianza requerida tendremos más asociaciones, como en el ejemplo siguiente.

In [6]:
H2 = apriori.generate_rules(F, soporte, min_confidence=0.7, verbose=True)

{Beer} ---> {Diapers}:  conf = 1.0, sup = 0.429
{Diapers} ---> {Bread}:  conf = 0.8, sup = 0.571
{Diapers} ---> {Milk}:  conf = 0.8, sup = 0.571
{Milk} ---> {Diapers}:  conf = 0.8, sup = 0.571
{Milk} ---> {Bread}:  conf = 0.8, sup = 0.571
{Milk, Bread} ---> {Diapers}:  conf = 0.75, sup = 0.429
{Diapers, Bread} ---> {Milk}:  conf = 0.75, sup = 0.429
{Diapers, Milk} ---> {Bread}:  conf = 0.75, sup = 0.429


## Un segundo ejemplo

Vamos a ver un ejemplo donde se generan reglas con más de un item en el antecendente para valores más altos de confianza.

In [7]:
dataset2 =  [['Soja', 'Zumo'], 
             ['Soja', 'Cereales','Leche'],
             ['Soja', 'Zumo','Galletas'],
             ['Soja', 'Galletas'],
             ['Soja', 'Cereales','Leche'],
             ['Soja', 'Cereales','Leche'],
             ['Soja', 'Cereales','Leche'],
             ['Soja', 'Cereales','Leche'],
             ['Cereales', 'Leche'],
             ['Cereales', 'Soja'],
             ['Cereales', 'Leche'],
             ['Soja', 'Leche', 'Galletas'],
             ['Cereales', 'Galletas'],
             ['Leche', 'Galletas', 'Zumo'],
             ['Leche', 'Soja', 'Zumo']
             ]


In [8]:
F2, soporte2 = apriori.apriori(dataset2, min_support=0.3)
H3 = apriori.generate_rules(F2, soporte2, min_confidence=0.8, verbose=True)

{Cereales, Soja} ---> {Leche}:  conf = 0.833, sup = 0.333


En este caso vemos que hay reglas como <code>{Cereales, Soja}-> {Leche}</code> que aún teniendo un soporte más pequeño, tienen una confianza muy alta. Concretamente:
* El 3-itemset aparece 5/15 veces, es decir, un 33%. Aunque pueda parecer bajo, habría que mirar el volumen del dataset. Si en un supermercado hay muchos productos, es posible que los productos comprados por clientes en una visita estén muy dispersos y exigir un soporte alto hará que el algoritmo no genere ninguna regla.
* Sin embargo, la combinación {Cereales, Soja} aparece 6 veces, de las cuales 5 aparece con Leche, es decir, 5/6 veces, un 83%, lo que representa un patrón muy significativo.

Es importante entender que el valor de confianza mínima tiene una influencia grande en el número de reglas, por lo que suele requerir algo de prueba y error. Por ejemplo:

In [9]:
H4 = apriori.generate_rules(F2, soporte2, min_confidence=0.7, verbose=True)

{Leche} ---> {Soja}:  conf = 0.7, sup = 0.467
{Cereales} ---> {Leche}:  conf = 0.778, sup = 0.467
{Leche} ---> {Cereales}:  conf = 0.7, sup = 0.467
{Cereales, Soja} ---> {Leche}:  conf = 0.833, sup = 0.333
{Leche, Soja} ---> {Cereales}:  conf = 0.714, sup = 0.333
{Leche, Cereales} ---> {Soja}:  conf = 0.714, sup = 0.333


Por último, es importante entender que las reglas quedan representadas en el resultado de <code>generate_rules()</code>, en nuestro caso en H.

Podemos tomar reglas particulares, el algoritmo nos devuelve una tupla con dos conjuntos y el valor de la confianza.

In [10]:
print H4[0]

(frozenset(['Leche']), frozenset(['Soja']), 0.7000000000000001)


## Comprendiendo la "poda"

Las posibles combinaciones de items son muchas, especialmente con datasets que tienen muchos items diferentes. Para tratar de que el algoritmo sea más eficiente, se utiliza un concepto de "poda" del espacio de posibilidades.

Vamos a ver el funcionamiento de esa poda, viendo algunas operaciones internas al algoritmo (es decir, son parte de la ejecución de la función <code>apriori</code> ya vista). Es decir, vamos a ejecutar "pasos internos" que antes se han ejecutado internamente al invocar al algoritmo.

Lo primero que hay hace el algoritmo es obtener los 1-itemsets candidatos. 

In [11]:
# Generación de los 1-itemsets.
C1 = apriori.create_candidates(dataset, verbose=True) 

{Beer, Bread, Coke, Diapers, Eggs, Milk}


A continuación, se "descartan" los 1-itemsets que no llegan al soporte mínimo establecido.

In [12]:
# La idea es eliminar de los 1-itemsets los que no lleguen al soporte establecido.
# Esto reduce las combinaciones que hay que examinar en los siguientes pasos.
# Como el soporte que pedimos es 60% vemos que se elimina "Coke" y "Eggs".
# Por ejemplo "Eggs" solo aparece en 1/5, es decir, un 20% de las transacciones.
# Este es un paso de "poda" (prune) de los candidatos.

ftemp, stemp = apriori.support_prune(dataset, C1, 0.6)
print "1-itemsets frecuentes:"
print ftemp
print "-----"
print "Soporte de los 1-itemsets:"
print stemp

1-itemsets frecuentes:
[frozenset(['Milk']), frozenset(['Bread']), frozenset(['Diapers'])]
-----
Soporte de los 1-itemsets:
{frozenset(['Eggs']): 0.14285714285714285, frozenset(['Diapers']): 0.7142857142857143, frozenset(['Beer']): 0.42857142857142855, frozenset(['Bread']): 0.8571428571428571, frozenset(['Coke']): 0.42857142857142855, frozenset(['Milk']): 0.7142857142857143}


A continuación, el algoritmo obtendría los 2-itemsets a partir de los 1-itemsets con suficiente soporte. Antes de seguir a los 3-itemsets, utilizará de nuevo <code>support_prune</code> para **tratar de reducir las combinaciones** que tendrá que dar en el siguiente paso.

Cabe hacer notar que si un 2-itemsets [A, B] tiene un soporte de digamos un 20%, cualquier 3-itemset al que añadamos cualquier elemento X, es decir, [A, B, X] tendrá un máximo de un 20% de soporte. Si nuestros soporte mínimo es 30%, buscar combinaciones [A, B, X] es inútil e ineficiente, ya que nunca superarán el 20%. 